In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Load the Excel file
file_path = r"D:\Akash\Work\AI\new_test_data_kaiseki\confidence_F1_curve_V360.xlsx"  # Replace with the actual file path
df = pd.read_excel(file_path, sheet_name="v360_test_dataset")

# Print the column names to identify the correct names
print("Column Names:", df.columns)

# Assuming the first column is for confidence scores and others are models
# Replace 'Confidence' if the actual column name is different
confidence_column = 'Confidence'  # Adjust based on actual column names
confidence_scores = df[confidence_column]

# Get the model columns, excluding the confidence column
model_columns = df.columns[df.columns != confidence_column]

# Initialize a dictionary to store AUC values
auc_scores = {}

# Calculate AUC for each model
for model in model_columns:
    # Extract model predictions
    model_predictions = df[model]
    
    # Ensure that the confidence values and model predictions are valid for AUC calculation
    if len(confidence_scores.unique()) > 1:
        # Compute the AUC using the confidence scores and model predictions
        auc = roc_auc_score(confidence_scores, model_predictions)
        auc_scores[model] = auc
    else:
        auc_scores[model] = None  # Handle case where AUC cannot be computed

# Convert the AUC scores to a DataFrame for better display
auc_df = pd.DataFrame(list(auc_scores.items()), columns=['Model', 'AUC'])

# Display the AUC table
print(auc_df)

# Optionally save the AUC scores to an Excel file
output_file_path = r"D:\Akash\Work\AI\new_test_data_kaiseki\analysis\auc_scores.xlsx"  # Replace with your desired output path
auc_df.to_excel(output_file_path, index=False)


In [22]:
import pandas as pd
import numpy as np

# Load the Excel file and specify the sheet name

df = pd.read_excel(file_path, sheet_name="v360_test_dataset")

# Extract confidence scores from column A
confidence_scores = df.iloc[:, 0]  # Assuming the first column (A) contains the confidence values

# Extract model outputs from column B onwards
model_columns = df.columns[1:]  # Columns B and onwards represent model outputs

# Initialize a dictionary to store AUC values
auc_scores = {}

# Iterate through each model and calculate AUC using np.trapz
for model in model_columns:
    # Extract model predictions
    model_predictions = df[model]
    
    # Combine confidence and predictions into a temporary DataFrame
    temp_df = pd.DataFrame({'confidence': confidence_scores, 'predictions': model_predictions})
    
    # Drop rows with NaN values
    temp_df = temp_df.dropna()

    # Sort values by confidence (x-axis)
    temp_df = temp_df.sort_values(by='confidence')
    
    # Use np.trapz to compute the area under the curve (AUC)
    auc = np.trapz(temp_df['predictions'], x=temp_df['confidence'])
    
    # Store the AUC for this model
    auc_scores[model] = auc

# Convert the AUC scores to a DataFrame for better display
auc_df = pd.DataFrame(list(auc_scores.items()), columns=['Model', 'AUC'])

# Display the AUC table
print(auc_df)

# Optionally save the AUC scores to an Excel file
auc_df.to_excel(output_file_path, index=False)


                                              Model       AUC
0                                           yolov5s  0.244085
1                                           yolov5m  0.249020
2                                           yolov5l  0.259974
3                                           yolov5x  0.275669
4              231216091234_runs_train_HBI_epoch100  0.520860
..                                              ...       ...
176  240918073603_runs_train_2024_09_tc4_dataset_13  0.601273
177   240922184434_runs_train_2024_09_tc4_dataset_1  0.590577
178   240922185231_runs_train_2024_09_tc4_dataset_3  0.583300
179   240922192458_runs_train_2024_09_tc4_dataset_2  0.590101
180   240923071239_runs_train_2024_09_tc4_dataset_4  0.588668

[181 rows x 2 columns]


In [24]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import ColorScaleRule

# Load the Excel file and specify the sheet name
# file_path = "/mnt/data/your_file.xlsx"  # Replace with actual file path
df = pd.read_excel(file_path, sheet_name="v360_test_dataset")

# Extract confidence scores from column A
confidence_scores = df.iloc[:, 0]  # Assuming the first column (A) contains the confidence values

# Extract model outputs from column B onwards
model_columns = df.columns[1:]  # Columns B and onwards represent model outputs

# Initialize a dictionary to store AUC values
auc_scores = {}

# Iterate through each model and calculate AUC using np.trapz
for model in model_columns:
    # Extract model predictions
    model_predictions = df[model]
    
    # Combine confidence and predictions into a temporary DataFrame
    temp_df = pd.DataFrame({'confidence': confidence_scores, 'predictions': model_predictions})
    
    # Drop rows with NaN values
    temp_df = temp_df.dropna()

    # Sort values by confidence (x-axis)
    temp_df = temp_df.sort_values(by='confidence')
    
    # Use np.trapz to compute the area under the curve (AUC)
    auc = np.trapz(temp_df['predictions'], x=temp_df['confidence'])
    
    # Store the AUC for this model
    auc_scores[model] = auc

# Convert the AUC scores to a DataFrame for better display
auc_df = pd.DataFrame(list(auc_scores.items()), columns=['Model', 'AUC'])

# Sort the AUC scores in descending order
auc_df = auc_df.sort_values(by='AUC', ascending=False)

# Display the sorted AUC table
print(auc_df)

# Save the AUC scores to an Excel file
output_file_path = r"D:\Akash\Work\AI\new_test_data_kaiseki\analysis\auc_scores_trapz_sorted.xlsx"
auc_df.to_excel(output_file_path, index=False)

# Apply conditional color coding using openpyxl
wb = load_workbook(output_file_path)
ws = wb.active

# Define a color scale (green for high AUC, red for low AUC)
color_rule = ColorScaleRule(
    start_type="percentile", start_value=0, start_color="FF6347",  # Red for low AUC
    mid_type="percentile", mid_value=50, mid_color="FFFF00",  # Yellow for mid AUC
    end_type="percentile", end_value=100, end_color="32CD32"  # Green for high AUC
)

# Apply the color rule to the AUC column
ws.conditional_formatting.add('B2:B{}'.format(len(auc_df)+1), color_rule)

# Save the workbook with color formatting
wb.save(output_file_path)

print(f"AUC scores saved to {output_file_path} with conditional color coding.")


                                                 Model       AUC
88   240704101619_runs_train_202403_7_BGs_2percent_ex1  0.700050
123     240727182157_runs_train_2024_07_tc1_dataset_26  0.621201
139      240803061121_runs_train_2024_07_tc1_dataset_4  0.621019
142      240803175810_runs_train_2024_07_tc1_dataset_7  0.620926
161     240901083931_runs_train_2024_08_tc3_dataset_16  0.615250
..                                                 ...       ...
2                                              yolov5l  0.259974
1                                              yolov5m  0.249020
0                                              yolov5s  0.244085
49            240524153431_runs_train_fisheye_overhead  0.217369
51                     240524182726_runs_train_WEPDTOF  0.199290

[181 rows x 2 columns]
AUC scores saved to D:\Akash\Work\AI\new_test_data_kaiseki\analysis\auc_scores_trapz_sorted.xlsx with conditional color coding.
